In [1]:
# import libraries

import ROOT
ROOT.gROOT.LoadMacro("stl_loader.h+")

from collections import defaultdict
import numpy as np
import pandas as pd
from IPython.display import Image

import hax

from lce_helpers import *

#hax.ipython.code_hider()

/project/lgrandi/anaconda3/envs/pax_head/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
# from 2015 Kr83m diffusion-mode data
datasets = ['xe100_150413_1839','xe100_150414_1535','xe100_150415_1749',
            'xe100_150416_1832','xe100_150419_1611','xe100_150420_0304',
            'xe100_150420_1809']

# xerawdp path
xerawdpPath = '/project/lgrandi/tunnell/run_14/NewNN/'

In [3]:
# get xerawdp tree
xerawdpTree = ROOT.TChain('T1')
t2 = ROOT.TChain('T2')
t3 = ROOT.TChain('T3')

for dataset in datasets:
    xerawdpTree.AddFile(xerawdpPath+dataset+'.root')
    t2.AddFile(xerawdpPath+dataset+'.root')
    t3.AddFile(xerawdpPath+dataset+'.root')
    
xerawdpTree.AddFriend(t2)
xerawdpTree.AddFriend(t3)
print(xerawdpTree.GetEntries())

5093213


In [6]:
cut_acceptance = np.zeros((8,2))
for i in range(xerawdpTree.GetEntries()):
    xerawdpTree.GetEntry(i)
    
    if xerawdpTree.NbS1Peaks < 2 or xerawdpTree.S1sTot[0] <= 0 or xerawdpTree.NbS2Peaks < 1:
        cut_acceptance[0,0] += 1
        continue
    if xerawdpTree.S1sCoin[0] < 2 or xerawdpTree.S1sCoin[1] < 2:
        cut_acceptance[1,0] += 1
    if xerawdpTree.cS2sTot[0] < 150:
        cut_acceptance[2,0] += 1
    if xerawdpTree.S1sLeftEdge[1]-xerawdpTree.S1sRightEdge[0] <= 0:
        cut_acceptance[3,0] += 1
    dt_s1 = (xerawdpTree.S1sPeak[1]-xerawdpTree.S1sPeak[0])*10
    if dt_s1 < 500 or dt_s1 > 1000:
        cut_acceptance[4,0] += 1
    if xerawdpTree.NbS2Peaks > 1 and xerawdpTree.cS2sTot[1] > 150:
        cut_acceptance[5,0] += 1
    if xerawdpTree.S2sRightEdge[0]-xerawdpTree.S2sLeftEdge[0] < xerawdpTree.S1sRightEdge[1]-xerawdpTree.S1sLeftEdge[0]:
        cut_acceptance[6,0] += 1
    if xerawdpTree.S1sTot[1]/xerawdpTree.S1sTot[0] < 0.1 or xerawdpTree.S1sTot[1]/xerawdpTree.S1sTot[0] > 1.0:
        cut_acceptance[7,0] += 1
    
print(cut_acceptance)

[[ 1657007.        0.]
 [ 1391466.        0.]
 [  151627.        0.]
 [ 1834915.        0.]
 [ 3170902.        0.]
 [  258139.        0.]
 [  890042.        0.]
 [ 2430306.        0.]]


In [ ]:
2.62*10.

In [ ]:
print(cut_acceptance)

In [ ]:
# Define and Apply 2 S1 and 1 S2 Xerawdp Event Selection

xerawdpTree.SetAlias('Xkr_2S1','NbS1Peaks >=2')
xerawdpTree.SetAlias('Xkr_1S2','NbS2Peaks == 1 || (NbS2Peaks > 1 && cS2sTot[1] > 150)')
xerawdpTree.SetAlias('Xkr_S1sCoin','S1sCoin[0] >= 2 && S1sCoin[1] >=2')
xerawdpTree.SetAlias('Xkr_S20Area','cS2sTot[0] >= 150')
xerawdpTree.SetAlias('Xkr_S1sPeakOverlap','S1sLeftEdge[1] - S1sRightEdge[0] > 0')
xerawdpTree.SetAlias('Xkr_S1Times','(S1sPeak[1]-S1sPeak[0])*10 >= 500 && (S1sPeak[1]-S1sPeak[0])*10 <= 1000')
xerawdpTree.SetAlias('Xkr_S1sAreaRatio','S1sTot[1]/S1sTot[0] > 0.1 && S1sTot[1]/S1sTot[0] < 1.0')
xerawdpTree.SetAlias('Xkr_S1SpanS2Width','S2sRightEdge[0]-S2sLeftEdge[0] >= S1sRightEdge[1]-S1sLeftEdge[0]')
xerawdpTree.SetAlias('Kr_2S1and1S2','Xkr_2S1 && Xkr_1S2 && Xkr_S1sCoin && Xkr_S20Area && Xkr_S1sPeakOverlap')

Lkr_2S1and1S2 = ROOT.TEventList("Lkr_2S1and1S2")
Lkr_2S1and1S2_file = ROOT.TFile("Lkr_2S1and1S2.root","RECREATE")
xerawdpTree.Draw(">>Lkr_2S1and1S2","Kr_2S1and1S2")
Lkr_2S1and1S2_file.Write()
Lkr_2S1and1S2_file.Close()

In [ ]:
Lkr_2S1and1S2_file = ROOT.TFile("Lkr_2S1and1S2.root","Read")
Lkr_2S1and1S2 = Lkr_2S1and1S2_file.Get("Lkr_2S1and1S2")
xerawdpTree.SetEventList(Lkr_2S1and1S2)
print('Kr83m 2S1 and 1S2: '+str(Lkr_2S1and1S2.GetN())+' '+str(Lkr_2S1and1S2.GetN()/xerawdpTree.GetEntries())+'\n')

data = defaultdict(list)

for i in range(Lkr_2S1and1S2.GetN()):
    xerawdpTree.GetEntry(Lkr_2S1and1S2.GetEntry(i))
    
    data['cs10Area'].append( xerawdpTree.cxS1sTot[0] )
    data['s10Area'].append( xerawdpTree.S1sTot[0] )
    data['s10Time'].append( xerawdpTree.S1sPeak[0] )
    data['s10Coin'].append( xerawdpTree.S1sCoin[0] )
    
    
    data['cs11Area'].append( xerawdpTree.cxS1sTot[1] )
    data['s11Area'].append( xerawdpTree.S1sTot[1] )
    data['s11Time'].append( xerawdpTree.S1sPeak[1] )
    data['s11Coin'].append( xerawdpTree.S1sCoin[1] )
    
    data['s10x'].append( xerawdpTree.cS2sPosNn[0][0] )
    data['s10y'].append( xerawdpTree.cS2sPosNn[0][1] )
    data['s10z'].append( xerawdpTree.cS2sPosNn[0][2] )
    
    data['s11x'].append( xerawdpTree.cS2sPosNn[0][0] )
    data['s11y'].append( xerawdpTree.cS2sPosNn[0][1] )
    data['s11z'].append( xerawdpTree.cS2sPosNn[0][2] )
    
    data['cs20Area'].append( xerawdpTree.cS2sTot[0] )
    
df_xerawdp = pd.DataFrame(data)

In [ ]:
print(df_xerawdp['cs20Area'])

In [ ]:
R = 15.25
Z = 30.3
A_r = [R/5, 2*R/5, 3*R/5, 4*R/5, R]
N_phi = [10, 15, 20, 40, 60]
N_z = 10.0
bin_settings = [R,Z,A_r,N_phi,N_z]

bin_data = xe100_to_lyBins(df_xerawdp,bin_settings,'s10',bin_spec_dir='Bin_Hists_s10_xerawdp')